In [1]:
import transformers, torch, os, numpy, sys
from transformers import AutoTokenizer, AutoModelForCausalLM
from time import time

import pandas

import sys
from llama_index.core import PromptTemplate
import os, pandas, datasets, json
from rouge_score import rouge_scorer

import rouge

from rouge import Rouge


import csv

/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
device = "cpu"
if torch.cuda.is_available():
    device = torch.cuda.current_device()
print(device)

cpu


In [3]:
textlist=[]
with open('xlxFile.csv', 'r', newline='') as csvfile:
    #Create a CSV reader object
    csv_reader = csv.reader(csvfile)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Each row will be treated as a list of strings
        # Convert it into a single string if needed
        row_string = ','.join(row)
        textlist.append(row_string)
        #print(row_string)

textlist[1]



section_criteriaS=["Permanent, paroxysmal, or persistent afib","CHADS2 Score","Prior stroke or transient ischemic attack","Arterial hypertension on treatment (high bp e.g. >140, or HTN)","Diabetes mellitus (DM1, DM2, T2D, T1DM, T2DM)","Max Blood Glucose","Max A1c","Heart Failure","Left ventricular ejection fraction (lowest mentioned, LVEF, ef, ejection fraction)","Valvular disease requiring or past surgery (stenosis)","Planned or past ablation procedure for afib","Serious bleeding in last 6 months","History Peptic Ulcer Disease","Minimum observed platelet count (PLT)","Minimum observed hemoglobin (HGB)","Stroke during admission or 10 days prior (Flag if recent, not clear 10 days)","Documented hemorrhagic tendencies or blood dyscrasias","Alcohol abuse - Generally not available in mimic iv - just list as -1","Drug abuse - Generally not available in mimic iv - just list as -1","max serum creatine  (Creat)","max aspartate aminotransferase level (AST)","max total bilirubin (TotBili, Bili)","Bipolar","Schizophrenic","Major depressive disorder / Depression","Inability to make medical decisions (Discharged - mental status / consciousness)"]
noteS=textlist[1]
system_prompt = """\
Your job is to decide which of the following inclusion criteria the given patient meets.
Below is a clinical note describing the patient's current health status:
---------------------
{note}
---------------------
The inclusion criteria being assessed are listed below:
{section_criteria}
---------------------
For each of the criteria above, use the patient's clinical note to determine whether the patient meets each criteria. Think step by step, and
justify your answer.
Format your response as a list with following elements with following type:
* Permanent, paroxysmal, or persistent afib: int , 1 if present, 0  if not present, -1 if no history to tell
* CHADS2 Score: int , a number between 0 to 10 , -1 if not present
* Prior stroke or transient ischemic attack: int , 1 if present, 0  if not present, -1 if no history to tell
* Arterial hypertension on treatment (high bp e.g. >140, or HTN): int , 1 if present, 0  if not present, -1 if no history to tell
* Diabetes mellitus (DM1, DM2, T2D, T1DM, T2DM): int , 1 if present, 0  if not present, -1 if no history to tell
* Max Blood Glucose: int , a number between o to 200, -1 if no level listed
* Max A1c: int , a number, -1 if no level listed
* Heart Failure: int , 1 if present, 0  if not present, -1 if no history to tell
* Left ventricular ejection fraction (lowest mentioned, LVEF, ef, ejection fraction): int , % if present, -1 if not in note, -2 if it was in note but filtered



* Valvular disease requiring or past surgery (stenosis): int , 1 if present, 0 if not present, -1 if no medical history
* Planned or past ablation procedure for afib: int , 1 if present, 0 if not present, -1 if no medical history
* Serious bleeding in last 6 months: int , 1 if present, 0 if not present, -1 if no medical history
* History Peptic Ulcer Disease: int , 1 if present, 0 if not present, -1 if no medical history
* Minimum observed platelet count (PLT): int , a number, -1 if no level listed
* Minimum observed hemoglobin (HGB): float , -1 if no level listed
* Stroke during admission or 10 days prior (Flag if recent, not clear 10 days): int , 1 if present, 0 if not present
* Documented hemorrhagic tendencies or blood dyscrasias: int , 1 if present, 0 if not present
* Alcohol abuse - Generally not available in mimic iv - just list as -1: int , a number, 1 if present, 0 if not present, -1 if not able to tell
* Drug abuse - Generally not available in mimic iv - just list as -1: int , a number, 1 if present, 0 if not present, -1 if not able to tell
* max serum creatine  (Creat): float, a number , -1 if no level listed
* max aspartate aminotransferase level (AST): int , a number , -1 if no level listed
* max total bilirubin (TotBili, Bili): float , a number, -1 if no level listed
* Bipolar: int , 1 if present, 0 if not present
* Schizophrenic: int , 1 if present, 0 if not present
* Major depressive disorder / Depression: int , 1 if present, 0 if not present
* Inability to make medical decisions (Discharged - mental status / consciousness): int , 1 if present, 0 if not present

An example of how your  response should be formatted is shown below:

Permanent, paroxysmal, or persistent afib: 1/0/-1
CHADS2 Score: a number
Prior stroke or transient ischemic attack: 1/0/-1
Arterial hypertension on treatment (high bp e.g. >140, or HTN): 1/0/-1
Diabetes mellitus (DM1, DM2, T2D, T1DM, T2DM): 1/0/-1
Max Blood Glucose: a number
Max A1c: a number
Heart Failure: 1/0/-1
Left ventricular ejection fraction (lowest mentioned, LVEF, ef, ejection fraction): a number



Valvular disease requiring or past surgery (stenosis): 1/0/-1
Planned or past ablation procedure for afib: 1/0/-1
Serious bleeding in last 6 months: 1/0/-1
History Peptic Ulcer Disease: 1/0/-1
Minimum observed platelet count (PLT): a number
Minimum observed hemoglobin (HGB): a number
Stroke during admission or 10 days prior (Flag if recent, not clear 10 days): 1/0
Documented hemorrhagic tendencies or blood dyscrasias: 1/0
Alcohol abuse - Generally not available in mimic iv - just list as -1: a number
Drug abuse - Generally not available in mimic iv - just list as -1: a number
max serum creatine  (Creat): a number
max aspartate aminotransferase level (AST): a number
max total bilirubin (TotBili, Bili): a number
Bipolar: 1/0
Schizophrenic: 1/0
Major depressive disorder / Depression: 1/0
Inability to make medical decisions (Discharged - mental status / consciousness): 1/0


The above example is only for illustration purposes only. It does not reflect the actual criteria or patient for this task.
Please analyze the given patient and inclusion criteria. Remember to include all inclusion criteria in your returned  list. Please
provide your response:
"""

prompt_tmpl = PromptTemplate(system_prompt)

partial_prompt_tmpl = prompt_tmpl.partial_format(tone_name="Shakespeare")
system_prompt = partial_prompt_tmpl.format(
    section_criteria=section_criteriaS,
    note=noteS,
)
print(system_prompt)

Your job is to decide which of the following inclusion criteria the given patient meets.
Below is a clinical note describing the patient's current health status:
---------------------
___ year old male with hx of HFpEF (EF 45-50% ___, STEMI s/p 
PCI (___) on ASA, PVD s/p L femoral-tibial (___) and R 
fem-pop bypass (___), CVA (___), CKD3, HTN, T2DM and BPH who 
presented to the ED on ___ with dyspnea and ___ swelling.

Earlier today (___), patient went to see PCP for ___ 
visit, had increased dyspnea, found to be hypoxic (O2sat in the 
___ on RA), and improved to 90 when placed on 2L NC. Patient was 
then sent to the ED. Patient reports having shortness of breath 
for the past two weeks. He has 10 lbs weight gain and increased 
leg swelling. Patient denies having chest pain, cough, 
orthopnea, or fevers. He was last hospitalized from 
___ because of acute decompensated HFpEF, hospital 
course c/b hypercarbic failure requiring intubation. Patient 
takes Lasix 60mg BID and spironolactone

In [4]:
import pandas

In [5]:
lama_size = '7B'
#drbench_dev_path = 'DrBench/Csv/summ_0821_dev.csv'
#model_path = "HuggingFaceH4/zephyr-7b-beta"
model_path = "togethercomputer/LLaMA-2-7B-32K"

if '7b' in model_path:
  os.environ['CUDA_VISIBLE_DEVICES'] = '0'
elif '13b' in model_path:
  os.environ['CUDA_VISIBLE_DEVICES'] = '1'
elif '70b' in model_path:
  os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'

In [6]:
def calc_rougel(generated_text, reference_text):
  """Compute Rouge-L score"""

  # {'rougeL': Score(precision=0.5, recall=0.6, fmeasure=0.5)}
  #scorer = rouge_scorer.RougeScorer(['rougeL'])
  #scorer = Rouge(metrics=['rougeL'])
  #scorer = rouge_scorer(['rougeL'])
  scorer = rouge_scorer.RougeScorer(['rougeL'])
  scores = scorer.score(reference_text, generated_text)
  f1 = scores['rougeL'].fmeasure

  return f1

In [7]:
def csv_to_zero_shot_data(data_csv_path, include_subjective=False):
  """Get summarization input/output pair tuples"""
  #print(data_csv_path)  
  df = pandas.read_csv(data_csv_path, dtype='str')

  # input/output pairs
  ios = []

  for assm, summ, subj in zip(df['Assessment'], df['Summary'], df['S']):
    if type(assm) == str and type(summ) == str and type(subj) == str:
      summ = summ.replace('#', '') # cleanup
      summ = summ.replace(':', '') # cleanup

      if include_subjective:
        input_text = f'### Subjective Section ###\n\n{subj}\n\n' \
                     f'### Assessment Section ###\n\n{assm}'
      else:
        input_text = f'### Assessment Section ###\n\n{assm}'

      prompt = f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>' \
               f'\n\n{input_text} [/INST]\n\n'

      ios.append((prompt, summ))

  return ios

In [8]:

dev_path = os.path.join("./xlxFile.csv")
inputs_and_outputs = csv_to_zero_shot_data(dev_path)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path) # run on cpu 

In [10]:
model = AutoModelForCausalLM.from_pretrained(
  model_path, use_cache=False)

# move model to current device 
model.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [02:25<00:00, 72.76s/it] 


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [21]:
f1s = []
inference_times = []
print("check")
for prompt_text, reference_output in inputs_and_outputs:
  print("check")
  start = time()
  # tokenize your texts and then return the ids of tokens 
  input_ids = tokenizer.encode(prompt_text, return_tensors="pt") # return torch.tensor 
  # "I love dog" -> [2, 150, 23] the shape of this is (3, ) but the shape should be (b, len_sentence)
  # (3, ) -> (1, 3) 
  # add empty axis to the place
  #input_ids = input_ids[None, :]
  # now your model is on gpu 
  # so your inputs to your model must be on gpu as well 
  input_ids = input_ids.to(device)
  
  generated_outputs = model.generate(input_ids, max_length=200, temperature=0.001)

  # transform it back to human-readible texts 
  output_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
  # generated_outputs = pipeline(
  #   prompt_text,
  #   do_sample=True,
  #   top_k=10,
  #   num_return_sequences=1,
  #   eos_token_id=tokenizer.eos_token_id,
  #   temperature=0.001,
  #   # Option 1: Increase max_length
  #   # max_length=1500  # Increase the limit to 1500 tokens

  #   # Option 2: Set max_new_tokens (recommended)
  #   max_new_tokens=500  # Generate a maximum of 500 new tokens after the input
  # )
  end = time()
  inference_times.append(end - start)



  print('************************************************\n')
  #print(generated_outputs[0]['generated_text'])
  print(output_text)
  print(f'\n### Reference Summary ###\n\n{reference_output}\n')
      # remove the the prompt from output and evaluate
  # end_index = generated_outputs[0]['generated_text'].index('[/INST]')
  # generated_text = generated_outputs[0]['generated_text'][end_index+7:]
  f1 =calc_rougel(output_text.lower(), reference_output.lower())
  f1s.append(f1)

av_inf_time = numpy.mean(inference_times)
print(f'\naverage inference time: {av_inf_time} seconds]')
print('average f1:', numpy.mean(f1s))

check
check


/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 3839, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


************************************************

[INST] <<SYS>>
Your job is to decide which of the following inclusion criteria the given patient meets.
Below is a clinical note describing the patient's current health status:
---------------------
___ year old male with hx of HFpEF (EF 45-50% ___, STEMI s/p 
PCI (___) on ASA, PVD s/p L femoral-tibial (___) and R 
fem-pop bypass (___), CVA (___), CKD3, HTN, T2DM and BPH who 
presented to the ED on ___ with dyspnea and ___ swelling.

Earlier today (___), patient went to see PCP for ___ 
visit, had increased dyspnea, found to be hypoxic (O2sat in the 
___ on RA), and improved to 90 when placed on 2L NC. Patient was 
then sent to the ED. Patient reports having shortness of breath 
for the past two weeks. He has 10 lbs weight gain and increased 
leg swelling. Patient denies having chest pain, cough, 
orthopnea, or fevers. He was last hospitalized from 
___ because of acute decompensated HFpEF, hospital 
course c/b hypercarbic failure requi

/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 3778, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


************************************************

[INST] <<SYS>>
Your job is to decide which of the following inclusion criteria the given patient meets.
Below is a clinical note describing the patient's current health status:
---------------------
___ year old male with hx of HFpEF (EF 45-50% ___, STEMI s/p 
PCI (___) on ASA, PVD s/p L femoral-tibial (___) and R 
fem-pop bypass (___), CVA (___), CKD3, HTN, T2DM and BPH who 
presented to the ED on ___ with dyspnea and ___ swelling.

Earlier today (___), patient went to see PCP for ___ 
visit, had increased dyspnea, found to be hypoxic (O2sat in the 
___ on RA), and improved to 90 when placed on 2L NC. Patient was 
then sent to the ED. Patient reports having shortness of breath 
for the past two weeks. He has 10 lbs weight gain and increased 
leg swelling. Patient denies having chest pain, cough, 
orthopnea, or fevers. He was last hospitalized from 
___ because of acute decompensated HFpEF, hospital 
course c/b hypercarbic failure requi

...

/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 3561, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


************************************************

[INST] <<SYS>>
Your job is to decide which of the following inclusion criteria the given patient meets.
Below is a clinical note describing the patient's current health status:
---------------------
___ year old male with hx of HFpEF (EF 45-50% ___, STEMI s/p 
PCI (___) on ASA, PVD s/p L femoral-tibial (___) and R 
fem-pop bypass (___), CVA (___), CKD3, HTN, T2DM and BPH who 
presented to the ED on ___ with dyspnea and ___ swelling.

Earlier today (___), patient went to see PCP for ___ 
visit, had increased dyspnea, found to be hypoxic (O2sat in the 
___ on RA), and improved to 90 when placed on 2L NC. Patient was 
then sent to the ED. Patient reports having shortness of breath 
for the past two weeks. He has 10 lbs weight gain and increased 
leg swelling. Patient denies having chest pain, cough, 
orthopnea, or fevers. He was last hospitalized from 
___ because of acute decompensated HFpEF, hospital 
course c/b hypercarbic failure requi

/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 3623, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


************************************************

[INST] <<SYS>>
Your job is to decide which of the following inclusion criteria the given patient meets.
Below is a clinical note describing the patient's current health status:
---------------------
___ year old male with hx of HFpEF (EF 45-50% ___, STEMI s/p 
PCI (___) on ASA, PVD s/p L femoral-tibial (___) and R 
fem-pop bypass (___), CVA (___), CKD3, HTN, T2DM and BPH who 
presented to the ED on ___ with dyspnea and ___ swelling.

Earlier today (___), patient went to see PCP for ___ 
visit, had increased dyspnea, found to be hypoxic (O2sat in the 
___ on RA), and improved to 90 when placed on 2L NC. Patient was 
then sent to the ED. Patient reports having shortness of breath 
for the past two weeks. He has 10 lbs weight gain and increased 
leg swelling. Patient denies having chest pain, cough, 
orthopnea, or fevers. He was last hospitalized from 
___ because of acute decompensated HFpEF, hospital 
course c/b hypercarbic failure requi

/home/nmollaei/.conda/envs/catii/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 3425, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [20]:
f1s = []
inferfence_times = []
print("check")
for prompt_text, reference_output in inputs_and_outputs:
  print("check")
  start = time()
  # tokenize your texts and then return the ids of tokens 
  input_ids = tokenizer.encode(prompt_text, return_tensors="pt") # return torch.tensor 
  # "I love dog" -> [2, 150, 23] the shape of this is (3, ) but the shape should be (b, len_sentence)
  # (3, ) -> (1, 3) 
  # add empty axis to the place
  #input_ids = input_ids[None, :]
  # now your model is on gpu 
  # so your inputs to your model must be on gpu as well 
  input_ids = input_ids.to(device)
  generated_outputs = model.generate(input_ids, max_length=200, temperature=0.001)



  output = generated_outputs
  # transform it back to human-readible texts 
  #output_text = tokenizer.decode(output[0], skip_special_tokens=True)
  output_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
  # generated_outputs = pipeline(
  #   prompt_text,
  #   do_sample=True,
  #   top_k=10,
  #   num_return_sequences=1,
  #   eos_token_id=tokenizer.eos_token_id,
  #   temperature=0.001,
  #   # Option 1: Increase max_length
  #   # max_length=1500  # Increase the limit to 1500 tokens

  #   # Option 2: Set max_new_tokens (recommended)
  #   max_new_tokens=500  # Generate a maximum of 500 new tokens after the input
  # )
  end = time()
  inference_times.append(end - start)



  print('************************************************\n')
  print(generated_outputs[0]['generated_text'])
  print(f'\n### Reference Summary ###\n\n{reference_output}\n')
      # remove the the prompt from output and evaluate
  # end_index = generated_outputs[0]['generated_text'].index('[/INST]')
  # generated_text = generated_outputs[0]['generated_text'][end_index+7:]
  f1 =calc_rougel(output_text.lower(), reference_output.lower())
  f1s.append(f1)

av_inf_time = numpy.mean(inference_times)
print(f'\naverage inference time: {av_inf_time} seconds]')
print('average f1:', numpy.mean(f1s))

check
check
************************************************



IndexError: too many indices for tensor of dimension 1

In [ ]:
f1s = []
inference_times = []
print("check")
for prompt_text, reference_output in inputs_and_outputs:
  print("check")
  start = time()
  encoding = tokenizer(prompt_text)
  print(encoding.keys())
  input_ids = encoding["input_ids"].to(device)
  attention_mask = encoding["attention_mask"].to(device)
  generated_outputs = model(
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    temperature=0.001,
    # Option 1: Increase max_length
    # max_length=1500  # Increase the limit to 1500 tokens

    # Option 2: Set max_new_tokens (recommended)
    max_new_tokens=500  # Generate a maximum of 500 new tokens after the input
  )
  end = time()
  inference_times.append(end - start)



  print('************************************************\n')
  print(generated_outputs[0]['generated_text'])
  print(f'\n### Reference Summary ###\n\n{reference_output}\n')
      # remove the the prompt from output and evaluate
  end_index = generated_outputs[0]['generated_text'].index('[/INST]')
  generated_text = generated_outputs[0]['generated_text'][end_index+7:]
  f1 =calc_rougel(generated_text.lower(), reference_output.lower())
  f1s.append(f1)

av_inf_time = numpy.mean(inference_times)
print(f'\naverage inference time: {av_inf_time} seconds]')
print('average f1:', numpy.mean(f1s))

check
check
dict_keys(['input_ids', 'attention_mask'])


AttributeError: 